In [ ]:
import os    
import pandas as pd
import ollama
import chromadb
import re
import sqlite3

In [15]:
def retrieveID(paper_name, cursor):
    connection = sqlite3.connect("papers.db")
    cursor = connection.cursor()

    query = """
    SELECT paper_id FROM papers
    WHERE title = ?
    Limit 1
    """
    cursor.execute(query, (paper_name,))
    results = cursor.fetchall()
    
    #This returns paperID for use in future queries because working with a single ID is easier than working with title string
    return(results[0][0])

    #retrieveID("Intersection Theory, Integrable Hierarchies and Topological Field Theory") Example use case returns 9201003

In [35]:
def get_similar_abstract(paper_id, cursor):
    #VectorDB is broken for me so I will assume this returns a list of IDs and try not to kill myself

    query = """
    SELECT abstract FROM papers
    WHERE paper_id = ?
    Limit 1
    """
    cursor.execute(query, (paper_name,))
    results = cursor.fetchall()
    #Retrieved abstract from relationalDB
    abstract = results[0][0]
    
    #Make Call to VectorDB; No clue if this works or not because once again my connection is being shot and stabbed
    #Instantiate Collection; using name = abstract change if named differently
    collection = client.get_or_create_collection(name="abstract")
    
    response = ollama.embeddings(
      prompt=abstract,
      model="mxbai-embed-large"
    )
    results = collection.query(
      query_embeddings=[response["embedding"]],
      n_results=30
    )

    #Data will have to be formatted by someone who can yk read the fucking data but just return list of ids
    data = results['ids'][0]
    return(data)

In [36]:
def get_similar_authors(paper_id, cursor):
    #VectorDB is broken for me so I will assume this returns a list of IDs and try not to kill myself

    query = """
    SELECT authors FROM papers
    WHERE paper_id = ?
    Limit 1
    """
    cursor.execute(query, (paper_name,))
    results = cursor.fetchall()
    #Retrieved abstract from relationalDB
    abstract = results[0][0]
    
    #Make Call to VectorDB; No clue if this works or not because once again my connection is being shot and stabbed
    #Instantiate Collection; using name = abstract change if named differently
    collection = client.get_or_create_collection(name="authors")
    
    response = ollama.embeddings(
      prompt=abstract,
      model="mxbai-embed-large"
    )
    results = collection.query(
      query_embeddings=[response["embedding"]],
      n_results=20
    )

    #Data will have to be formatted by someone who can yk read the fucking data but just return list of ids
    data = results['ids'][0]
    return(data)

In [ ]:
def find_co_citations(paper_id, cursor, limit = 30):

    #This query returns top co-citations as a list of tuples [(paper_id, citation_count)] 
    # query = """
    # SELECT 
    #     CASE 
    #         WHEN paper_1_id = ? THEN paper_2_id
    #         ELSE paper_1_id 
    #     END AS co_cited_paper_id,
    #     co_citation_count
    # FROM co_citations
    # WHERE paper_1_id = ? OR paper_2_id = ?
    # ORDER BY co_citation_count DESC
    # """
    query = """
    SELECT 
        CASE 
            WHEN paper_1_id = ? THEN paper_2_id
            ELSE paper_1_id 
        END AS co_cited_paper_id,
        co_citation_count,
        pd.publication_date
    FROM co_citations
    JOIN paper_dates pd ON 
        (pd.paper_id = paper_1_id OR pd.paper_id = paper_2_id)
    WHERE paper_1_id = ? OR paper_2_id = ?
    ORDER BY co_citation_count DESC, pd.publication_date DESC
    LIMIT ?
    """
    cursor.execute(query, (paper_id, paper_id, paper_id, limit))
    top_co_citations = (cursor.fetchall())
    #print(top_co_citations)
    paper_id_list = [x[0] for x in top_co_citations]
    return paper_id_list
    #Sample Input: 
    #find_co_citations(9201001)
    #Sample Output: ['9201013', '9302014', '9203009', '9201011', '9201033', '9203043',
    #'9208031', '9208046', '9302048', '9303139', '9304011', '9201003', '9202006', '9203030',
    #'9206090', '9307063', '9312210', '9505127', '9201010', '9207020']

In [ ]:
def filter_recommendation(paper_ids, cursor):
    query = """
    SELECT title from papers
    WHERE id in ?
    ORDER BY citation_count DESC, submitted_date DESC
    LIMIT 10
    """
    cursor.execute()
    retList = cursor.fetchall()
    return retList

In [ ]:
#Use abstract of new paper inside vector db to get most similar papers,
#also use another collection with author names and return top similar queries from that.
#Use the ids of these papers and co-citations (find other papers co-cited with input), 
#and further filter them inside relational database using # of citations, recency,
#Return final ids
def reccPaper(paper_name):
    #Connect to SQL
    connection = sqlite3.connect("papers.db")
    cursor = connection.cursor()

    #Retrieve paper id for further querying
    paper_id = retrieveID(paper_name, cursor)
    
    #Return list of IDs from abstract via VectorDB
    abstract_id_list = get_similar_abstract(paper_id, cursor)
    
    #Collection of similar to author names
    author_id_list = get_similar_authors(paper_id, cursor)

    #Relational DB: Retrieve list of co-citations for further filtering
    co_citation_list = find_co_citations(paper_id, cursor)
    
    #Expected Output at this point: 3 lists of paper_ids
    #Option 1 Combine abstract and author list
    combined_list = abstract_id_list + author_id_list + co_citation_list
    
    #Option 2 Only use papers that appear in both lists
    #intersect_list = [x for x in abstract_id_list if x in author_id_list]
    #unique_list = list(set(abstract_id_list).symmetric_difference(set(author_id_list)))

    final_recommendation_list = []
    #Iterate through co_citations from relational DB; if id appears in co-citations append to final output list
    
    
    #End Connection
    connection.commit()
    cursor.close()
    connection.close()
    
    #Return Final IDs
    return final_recommendation_list